Test 9: Digital Logic Threshold Verification (VIH/VIL Margin)

Objective: Verify the robustness of the I2C interface by finding the breaking point of the logic levels. We will determine if the sensor complies with the VIH (Input High) Specification.

In [ ]:
from ctypes import *
import time
from sys import path
from os import sep
import numpy as np

dwf = cdll.LoadLibrary("libdwf.so")
constants_path = "/usr/share/digilent/waveforms/samples/py"
path.append(constants_path)
import dwfconstants as constants

In [ ]:
dwf.FDwfDeviceCloseAll()
filter_flags = c_int(constants.enumfilterType.value | constants.enumfilterUSB.value)
device_count = c_int()
dwf.FDwfEnum(filter_flags, byref(device_count))

hdwf = c_int()
dwf.FDwfDeviceOpen(c_int(-1), byref(hdwf))
if hdwf.value == 0:
    raise RuntimeError("Failed to open device.")
print(hdwf.value)

dwf.FDwfAnalogIOReset(hdwf)
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(1), c_double(3.0)) #setting VDD = 3V
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(0), c_double(1))
dwf.FDwfAnalogIOEnableSet(hdwf, c_int(1))
time.sleep(0.5)

vpos = c_double()
dwf.FDwfAnalogIOStatus(hdwf)
dwf.FDwfAnalogIOChannelNodeStatus(hdwf, c_int(0), c_int(1), byref(vpos))
print(f"Power: +{vpos.value:.2f}V")

In [ ]:
DEVICE_ADDR = 0x52
ENABLE_REG = 0x80
ATIME_REG = 0x81
CONTROL_REG = 0x8F
CDATAL_REG = 0x94
CDATAH_REG = 0x95
RDATAL_REG = 0x96
RDATAH_REG = 0x97
GDATAL_REG = 0x98
GDATAH_REG = 0x99
BDATAL_REG = 0x9A
BDATAH_REG = 0x9B

def write_register(reg, value):
    iNak = c_int()
    rgTX = (c_ubyte * 2)(reg, value)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(2), byref(iNak))
    return iNak.value == 0

def read_register(reg):
    iNak = c_int()
    rgTX = (c_ubyte * 1)(reg)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(1), byref(iNak))
    if iNak.value == 0:
        rgRX = (c_ubyte * 1)()
        dwf.FDwfDigitalI2cRead(hdwf, c_int(DEVICE_ADDR), rgRX, c_int(1), byref(iNak))
        if iNak.value == 0:
            return rgRX[0]
    return None

dwf.FDwfDigitalI2cRateSet(hdwf, c_double(100e3))
dwf.FDwfDigitalI2cSclSet(hdwf, c_int(0))
dwf.FDwfDigitalI2cSdaSet(hdwf, c_int(1))
iNak = c_int()
dwf.FDwfDigitalI2cClear(hdwf, byref(iNak))

In [ ]:
write_register(ENABLE_REG, 0x01) #Power on / PON = 1
time.sleep(0.01)
write_register(ENABLE_REG, 0x03) #Now, also RGBC enable / PON = 1, AEN = 1
write_register(ATIME_REG, 0xF6) #To run for 24ms, upto a max count of 10,240
write_register(CONTROL_REG, 0x02) #Setting AGAIN=16

In [ ]:
#Function to test the sensor output
def read_clear():
    time.sleep(0.1)
    
    clear_low = read_register(CDATAL_REG)
    clear_high = read_register(CDATAH_REG)
    
    if all(val is not None for val in [clear_low, clear_high]):
        clear = (clear_high << 8) | clear_low
        return {'clear': clear}
    return None

In [ ]:
#Fixing the sensor supply
SENSOR_VDD = 3.0
VIH_LIMIT = 0.7 * SENSOR_VDD # 2.1V
print(f"Sensor VDD Fixed at: {SENSOR_VDD}V")
print(f"Theoretical VIH Limit: {VIH_LIMIT:.2f}V (Communication should fail below this)")

logic_voltages = np.arange(3.3, 1.1, -0.1)

print("\n Starting logic level sweep... \n")

for v_logic in logic_voltages:
    dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(1), c_double(v_logic))
    dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(0), c_double(1)) # Enable

chip_id = read_register(0x92) #ID register

status = "PASS"
hex_id = "N/A"

if chip_id is None:
    status = "FAIL (NACK)"
elif chip_id != 0x44:
    status = "FAIL (Wrong ID)"
    hex_id = hex(chip_id)
else:
    hex_id = hex(chip_id)
print(f"Logic V: {v_logic:.2f}V | Chip ID: {hex_id} | Status: {status}")

if chip_id is None or chip_id != 0x44:
    if v_logic < VIH_LIMIT:
        print("  Expected failure")
    else:
        print("  Unexpected communication failure!")

print("\nTest Complete.")
print("Note: If communication fails around 2.1V, the sensor complies with the datasheet.")

print("\nRestoring default conditions...")
time.sleep(0.5)
dwf.FDwfDeviceCloseAll()
print("Device closed.")